In [104]:
import numpy as np
import sympy as sp
from random import random
from numpy import linalg
from scipy.linalg import expm
from scipy.linalg import inv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Optimal temporal energy growth

The optimization problem can therefore be formalized in the following way:
$$
F    = \dfrac{dq}{dt} - N(q) = 0 \\
F_0  = q(0) - g = 0 \\
\mathcal{J} = \dfrac{g \cdot g}{q(T)\cdot q(T)}
$$
Forming the Lagrangian and following the variational principle leads to the following sets of equations for the forward, backward and optimality conditions respectively, 
$$
\dfrac{dq}{dt} = N(q), \;\;q(0) = g \\
-\dfrac{da}{dt} = \left(\dfrac{\partial N}{\partial q} \right)^\top \cdot a, \;\; a(T) = - 2q(T)\dfrac{g \cdot g}{(q(T) \cdot q(T))^2} \\
g = a(0) \dfrac{q(T) \cdot q(T)}{2}
$$

For the specific problem of optimal transient growth an iterative method can be used where the three equations of the optimality system are solved sequentially. The method is initialized by giving an initial guess on $g$, then the loop is as follows:

- Given the $p^{th}$ guess $g^p$, compute the solution of the state equation integrating it forward in time from $t = 0$ to $t = T$ with initial condition $q(0) = g^p$.
- Compute $\mathcal{J}$ and its relative increment. If convergence is reached, stop, else continue.
- Use $q(T)$ to compute $a(T)$ and integrate the adjoint system backward in time from $t = T$ to $t = 0$.
- Use $a(0)$ to update the control using the optimality condition and get $g^{(p+1)}$. Then go to top.

Apply this algorithm to compute the optimal transient growth for the linear problem with : dq/dt = Lq

In [106]:
# define the state equation
def StateForw(t,q,L) :
    f = L*q
    return f

In [107]:
# define Adjoint equation rhs
def AdjntBack(t,a,L) : 
    f = L.transpose()*a
    return f


In [108]:
# Define the system


Rey = 400.0
T   = 200.0
L   = np.array([[1.0/Rey, 0.0], 
       [1.0, -3.0/Rey]])                  #>>> the opperator is linear here so dq/dt = Lq

# define the exact solution using the norm function

G_exact = (linalg.norm(expm(L*T)))**2     #>>> the exact equation is computed y = y0exp(Lt)

In [109]:
# Define tolerance and initialize iterations
tol   = 10**(-8);
g     = np.array([[random()], [random()]]) # (random initial guess) 
J     =  10**23
dJrel =  10**23 
it    =  0

In [110]:
# Iteration loop 
while (dJrel > tol) : 
    it   = it+1
    Jold = J
    Pdir = expm(L*T) "CFD"                   # (propagato the forward problem from 0 to T)
    qT   = np.dot(Pdir,g)               # (solve state equation forward in time) 
    g2   = np.dot(g.transpose(),g) 
    qT2  = np.dot(qT.transpose(),qT)
    J    = g2/qT2                       # (cost function)
    dJrel= abs((J-Jold)/J)
    aT   = -2.0*qT*(g2)/qT2**2          # (IC for adjoint equation) 
    Padj = inv(expm(T*L.transpose())) "adjoint solver"  #(adjoint propagator from T to 0) 
    a0   = np.dot(Padj,aT)
    g    = a0*(qT2/2.0);                # (optimality equation)
    g    = g/math.sqrt(np.dot(g.transpose(),g)) 
    
# end of iteration loop 
G = 1.0/J

In [127]:
# print results
print('number of iterations to convergence = ', it)
print('The resulting Gain computed = ', G)
print('The exact Gain              = ', G_exact)
print('optimal intial condition, g = ', g.transpose())
print('optimal response        , qT= ', qT.transpose())


number of iterations to convergence =  3
The resulting Gain computed =  [[20322.3643793]]
The exact Gain              =  20325.86821373707
optimal intial condition, g =  [[ 0.99993313 -0.01156461]]
optimal response        , qT=  [[  1.64861102 142.54699738]]
